# NEO DATA MANAGER

In [190]:
import requests 
import json 
import pandas as pd
from pandas.io.json import json_normalize

API_URL = 'https://api.nasa.gov/neo/rest/v1/feed'
API_PARAMS = '?start_date={}&end_date={}&api_key={}'

start_date = '2019-09-06'
end_date = '2019-09-07'

api_url = API_URL + API_PARAMS.format(start_date,end_date,'KwRDL3x7w75naVBiJua5GmSpMEDn75yNMjMqacS8')

response=requests.get(api_url)
response.raise_for_status()
neodata=json.loads(response.text)

# Approach 1

#### It is much faster than approach 2 but requires modification to access close approach data

In [242]:
program_starts = time.time()
new_df = pd.concat([pd.DataFrame(json_normalize(neodata['near_earth_objects'],x,max_level=10)) for x in neodata['near_earth_objects'].keys()]).reset_index(drop=True)
now = time.time()
print("It has been {0} seconds since the loop started".format(now - program_starts))


print(new_df.shape)
#print(json.dumps(neodata, sort_keys=True, indent=4))
new_df.head(2)

It has been 0.01296687126159668 seconds since the loop started
(25, 17)


,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,3840850,3840850,(2019 GT3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3840850,20.975,True,"[{'close_approach_date': '2019-09-06', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3840850?ap...,0.169650,0.379350,169.650435,379.349904,0.105416,0.235717,556.595932,1244.58634
1,2465617,2465617,465617 (2009 EK1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2465617,21.400,True,"[{'close_approach_date': '2019-09-06', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/2465617?ap...,0.139494,0.311918,139.493823,311.917671,0.086677,0.193817,457.656914,1023.35197


# Approach 2

##### Is able to access all the required data but is much slower than Approach 2

In [241]:
print("Today " + str(neodata["element_count"]) + " asteroids will be passing close to planet Earth:")
print("")
asteroids = neodata["near_earth_objects"]
print(asteroids.keys())

tmp = pd.DataFrame()
program_starts = time.time()
for asteroid in asteroids:
    for field in asteroids[asteroid]:
            tmp = tmp.append({'Estimated_Diameter': round((field["estimated_diameter"]["meters"]["estimated_diameter_min"]+field["estimated_diameter"]["meters"]["estimated_diameter_max"])/2),
                            'Close_Approach_Date_Time': field["close_approach_data"][0]["close_approach_date_full"],
                            'Velocity':field["close_approach_data"][0]["relative_velocity"]["kilometers_per_hour"],
                            'Distance_to_Earth':field["close_approach_data"][0]["miss_distance"]["kilometers"], 
                            'Asteroid_name':field["name"],
                            'Hazardous':field["is_potentially_hazardous_asteroid"],
                            'neo_ref_id':field["id"]},ignore_index=True)
now = time.time()
print("It has been {0} seconds since the loop started".format(now - program_starts))
tmp.head()

Today 25 asteroids will be passing close to planet Earth:

dict_keys(['2019-09-06', '2019-09-07'])
It has been 0.14963793754577637 seconds since the loop started


,Asteroid_name,Close_Approach_Date_Time,Distance_to_Earth,Estimated_Diameter,Hazardous,Velocity,neo_ref_id
0,(2019 GT3),2019-Sep-06 04:21,7474531.957218561,275.0,1.0,49109.3895423964,3840850
1,465617 (2009 EK1),2019-Sep-06 05:15,8553741.941962645,226.0,1.0,25847.5000908791,2465617
2,(2011 EE41),2019-Sep-06 00:08,37535522.259869113,59.0,0.0,33567.6912462274,3559741
3,(2013 SN24),2019-Sep-06 06:24,61382586.639217287,43.0,0.0,65918.9594914585,3648711
4,(2017 SH12),2019-Sep-06 03:17,36772337.984937047,75.0,0.0,45095.8604887573,3781726
